In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Asthma"
cohort = "GSE184382"

# Input paths
in_trait_dir = "../../input/GEO/Asthma"
in_cohort_dir = "../../input/GEO/Asthma/GSE184382"

# Output paths
out_data_file = "../../output/preprocess/Asthma/GSE184382.csv"
out_gene_data_file = "../../output/preprocess/Asthma/gene_data/GSE184382.csv"
out_clinical_data_file = "../../output/preprocess/Asthma/clinical_data/GSE184382.csv"
json_path = "../../output/preprocess/Asthma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Sputum microRNA‐screening reveals Prostaglandin EP3 receptor as selective target in allergen‐specific immunotherapy"
!Series_summary	"Several microRNAs (miRs) have been described as potential biomarkers in liquid biopsies and in the context of allergic asthma, while therapeutic effects on the airway expression of miRs remain elusive. In this study, we investigated epigenetic miR-associated mechanisms in the sputum of grass pollen allergic patients with and without allergen specific immunotherapy (AIT). Induced sputum samples of healthy controls (HC), AIT treated and untreated grass pollen allergic rhinitis patients with (AA) and without asthma (AR) were profiled using miR microarray and transcriptome microarray analysis of the same samples. miR targets were predicted in silico and used to identify inverse regulation. Local PGE2 levels were measured using ELISA."
!Series_summary	"Two Hundred and fifty nine miRs were upregulated in the sputum of AA 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Step 1: Analyze if gene expression data is likely to be available
is_gene_available = False  # This dataset appears to be focused on microRNA profiling, not gene expression

# Step 2: Determine availability and data conversion for trait, age, and gender
# Looking at the sample characteristics, we see:
# {0: ['season: in season'], 1: ['ait treatment: yes', 'ait treatment: no']}

# 2.1 Data Availability
# For trait (Asthma), the data can be inferred from the background mentioning "AA" (allergic asthma)
# and "AR" (allergic rhinitis without asthma), but not directly available in the sample characteristics
trait_row = None  # Asthma status not directly available in the sample characteristics
age_row = None    # Age information not available
gender_row = None # Gender information not available

# 2.2 Data Type Conversion Functions
# Since trait data is not available, we still define a conversion function but it won't be used
def convert_trait(value):
    if value is None:
        return None
    value = value.lower().split(':', 1)[-1].strip()
    if 'asthma' in value or 'aa' in value:
        return 1
    elif 'no asthma' in value or 'ar' in value:
        return 0
    return None

def convert_age(value):
    if value is None:
        return None
    try:
        # Extract numeric value after colon
        age_str = value.split(':', 1)[-1].strip()
        return float(age_str)
    except:
        return None

def convert_gender(value):
    if value is None:
        return None
    value = value.lower().split(':', 1)[-1].strip()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    return None

# Step 3: Save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Step 4: Clinical Feature Extraction
# Since trait_row is None, we'll skip this substep


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Asthma/GSE184382/GSE184382_series_matrix.txt.gz


Gene data shape: (58341, 39)
First 20 gene/probe identifiers:
Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '3xSLv1', 'A_19_P00315452',
       'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315502', 'A_19_P00315506',
       'A_19_P00315518', 'A_19_P00315519', 'A_19_P00315529', 'A_19_P00315541'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the gene expression data
# The identifiers like "(+)E1A_r60_1", "A_19_P00315452" are not standard human gene symbols
# These appear to be Agilent microarray probe IDs that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872', 'A_33_P3267760'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_001105533', nan], 'GB_ACC': [nan, nan, nan, 'NM_001105533', nan], 'LOCUSLINK_ID': [nan, nan, nan, 79974.0, 54880.0], 'GENE_SYMBOL': [nan, nan, nan, 'CPED1', 'BCOR'], 'GENE_NAME': [nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1', 'BCL6 corepressor'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.189652', nan], 'ENSEMBL_ID': [nan, nan, nan, nan, 'ENST00000378463'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673', 'ens|ENST00000378463'], 'CHROMOSOMAL_LOCATION': [nan, nan, 'unmapped', 'chr7:120901888-120901947', 'chrX:39909128-39909069'], 'CYTOBAND': [nan, nan, nan, 'hs|7q31.31', 'hs|Xp11.4'], 'DESCRIPTION': [nan, nan, nan, 'Homo sapiens cadherin-like and PC-esterase domain containing 1 (CPED1), transcript variant 2, mRNA [NM_001105533

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify relevant columns for mapping
# Looking at the gene annotation preview:
# 'ID' appears to contain identifiers similar to those in gene_data index (e.g., A_19_P... format)
# 'GENE_SYMBOL' contains the human gene symbols we want to map to

# 2. Extract gene mapping from annotation data
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')
print(f"Gene mapping shape: {mapping_df.shape}")
print("First few rows of the mapping dataframe:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Processed gene_data shape: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10])


Gene mapping shape: (48862, 2)
First few rows of the mapping dataframe:
               ID    Gene
3   A_33_P3396872   CPED1
4   A_33_P3267760    BCOR
5    A_32_P194264   CHAC2
6    A_23_P153745   IFI30
10  A_21_P0014180  GPR146


Processed gene_data shape: (29222, 39)
First few gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A1CF-2', 'A1CF-3', 'A2M', 'A2M-1',
       'A2M-AS1', 'A2ML1', 'A2MP1'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Since trait_row was determined to be None in Step 2, the dataset lacks the trait information
# we need for our analysis. We'll save the gene data but won't attempt to create linked data.

# Create a minimal dataframe with the trait column
dummy_df = pd.DataFrame({trait: []})

# 5. Validate and save cohort information - with final assessment
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,  # We do have gene expression data
    is_trait_available=False,  # But we lack the trait information
    is_biased=True,  # A dataset without trait information is considered biased for our purposes
    df=dummy_df,  # Minimal dataframe with trait column
    note="Dataset contains gene expression data but lacks explicit Asthma trait information in the sample characteristics."
)

print("Dataset is not usable for analysis due to missing trait information. Gene data was saved, but no linked data file was created.")

Normalized gene data saved to ../../output/preprocess/Asthma/gene_data/GSE184382.csv
Abnormality detected in the cohort: GSE184382. Preprocessing failed.
Dataset is not usable for analysis due to missing trait information. Gene data was saved, but no linked data file was created.
